In [1]:
from datasets import LogisticRegressionSameWeightsSampleLabelsDataset
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NUM_FEATURES = 4
data = LogisticRegressionSameWeightsSampleLabelsDataset(num_points=1000, num_features=NUM_FEATURES)

In [3]:
data.true_weights # TODO: try this experiment with the same weights for each feature, also maybe the gaussian. Also try sampling from the logit probability instead of just picking >0.5

array([1., 1., 1., 1.])

In [4]:
1 / (1 + np.exp(-data.train_data.dataset.tensors[0][:10] @ data.true_weights))

tensor([0.9954, 0.8441, 0.8705, 0.8403, 0.6787, 0.1447, 0.6624, 0.9717, 0.0448,
        0.8511], dtype=torch.float64)

In [5]:
# 0th index is data, 1xt index is label
data.train_data.dataset.tensors[0][:10], 1 / (1 + np.exp(-data.train_data.dataset.tensors[0][:10] @ data.true_weights)), data.train_data.dataset.tensors[1][:10]

(tensor([[ 1.7641,  0.4002,  0.9787,  2.2409],
         [ 1.8676, -0.9773,  0.9501, -0.1514],
         [-0.1032,  0.4106,  0.1440,  1.4543],
         [ 0.7610,  0.1217,  0.4439,  0.3337],
         [ 1.4941, -0.2052,  0.3131, -0.8541],
         [-2.5530,  0.6536,  0.8644, -0.7422],
         [ 2.2698, -1.4544,  0.0458, -0.1872],
         [ 1.5328,  1.4694,  0.1549,  0.3782],
         [-0.8878, -1.9808, -0.3479,  0.1563],
         [ 1.2303,  1.2024, -0.3873, -0.3023]], dtype=torch.float64),
 tensor([0.9954, 0.8441, 0.8705, 0.8403, 0.6787, 0.1447, 0.6624, 0.9717, 0.0448,
         0.8511], dtype=torch.float64),
 tensor([1, 1, 1, 1, 1, 0, 0, 1, 0, 1]))

In [6]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 0    364
1    336
dtype: int64
Val data:
 0    51
1    49
dtype: int64
Test data:
 1    103
0     97
dtype: int64


In [7]:
train_X, train_y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=len(data.train_data))))
test_X, test_y = next(iter(torch.utils.data.DataLoader(data.test_data, batch_size=len(data.test_data))))
train_X, train_y

(tensor([[-0.8034, -0.6895, -0.4555,  0.0175],
         [-1.3159, -0.4616, -0.0682,  1.7133],
         [-1.9138, -2.2855, -1.6018, -0.0379],
         ...,
         [-0.2070, -0.5633,  0.6772, -0.0319],
         [-0.6374, -0.3973, -0.1329, -0.2978],
         [ 0.1420, -0.3193,  0.6915,  0.6947]], dtype=torch.float64),
 tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
         1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
         1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
         1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
         0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
         0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
         1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
         0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
         0, 0, 0, 1, 1, 0, 0, 0, 

In [8]:
from sklearn.linear_model import LogisticRegression
m = LogisticRegression(fit_intercept=False)
m.fit(X=train_X, y=train_y)
m.score(X=test_X, y=test_y)

0.775

In [9]:
m.coef_

array([[0.97449704, 1.16692995, 1.12751498, 0.86434327]])

In [10]:
test_X[:10], test_y[:10]

(tensor([[ 9.4725e-01, -1.5501e-01,  6.1408e-01,  9.2221e-01],
         [-9.3898e-01,  1.0170e+00,  1.4230e+00,  3.9609e-01],
         [ 3.6585e-01,  1.2978e+00,  4.8112e-01,  2.7594e+00],
         [-7.8910e-01,  1.2485e-03, -1.5994e-01, -8.3196e-01],
         [ 1.9229e+00,  1.4805e+00,  1.8676e+00,  9.0604e-01],
         [-3.2493e-01, -7.1131e-01, -3.8815e-01, -5.9928e-02],
         [ 2.2805e-01,  2.0148e-01,  5.4077e-01, -1.8181e+00],
         [ 9.3680e-01,  7.7033e-01,  1.4081e-01,  4.7349e-01],
         [ 6.6559e-01,  3.6163e-03,  1.5467e+00,  2.1441e-01],
         [-3.7678e-01, -9.0903e-01, -1.4517e-01, -5.3633e-01]],
        dtype=torch.float64),
 tensor([0, 0, 1, 0, 1, 0, 1, 1, 1, 0]))

In [11]:
np.array_equal(
    (1 / (1 + np.exp(-test_X[:10] @ m.coef_.T)) > 0.5).view(-1), 
    test_y[:10]
)

False

In [12]:
# from brunoflow.net import LogReg
# m_bf = LogReg(input_size=NUM_FEATURES, bias=False)
# m_bf.